# Coalition-Finding Algorithmus

In diesem Notebook demonstrieren wir die Funktionsweise eines heuristischen Algorithmus zur Koalitionssuche,
der darauf abzielt, innerhalb einer gegebenen Menge von Interaktionswerten die *beste* bzw. *schlechteste* Koalition
festgelegter Größe zu identifizieren.

Jede Koalition \\( S \subseteq N \\) mit fester Größe \\( |S| = l \\) wird anhand ihrer aggregierten Interaktionswerte bewertet.
Diese Bewertung erfolgt über eine Funktion \\( \hat{\nu}_e(S) \\), welche alle zu \\( S \\) gehörenden Interaktionen aufsummiert:

\[
\hat{\nu}_e(S) := e_0 + \sum_{i \in S} e_{\{i\}} + \sum_{\{i,j\} \subseteq S} e_{\{i,j\}} + \sum_{\substack{T \subseteq S \\\\ |T| > 2}} e_T
\]

Der hier vorgestellte `subset_finding` Algorithmus nutzt eine Beam-Search-Heuristik zur effizienten Suche solcher Koalitionen.
Wir zeigen dies im Folgenden anhand eines Beispiels mit echten Interaktionswerten.




## Vorbereitung: Imports

Wir beginnen mit dem Import aller benötigten Module für Modellierung, Datenaufbereitung und den Koalitionsfindungs-Algorithmus.



In [1]:
from __future__ import annotations

from shapiq.datasets import load_bike_sharing
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

## Datensatz und Regressionsmodell

Wir verwenden den `bike_sharing`-Datensatz, um ein Random-Forest-Regressionsmodell zu trainieren.
Der Datensatz enthält numerische Merkmale (z.B. Temperatur, Luftfeuchtigkeit, Uhrzeit),
mit denen die Anzahl ausgeliehener Fahrräder vorhergesagt werden soll.


In [2]:
# Bike-Sharing-Daten laden
X, y = load_bike_sharing()

# Wir wählen 16 zufällige Datenpunkte für die spätere Koalitionsanalyse
X = X.sample(n=16, random_state=42)
y = y.loc[X.index]

# Trainingsdaten aufteilen und Modell trainieren
X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=1)
model = RandomForestRegressor().fit(X_train, y_train)

## Interaktionswerte berechnen

Um sinnvolle Koalitionen identifizieren zu können, benötigen wir sogenannte Interaktionswerte.
Diese quantifizieren den Beitrag einzelner oder kombinierter Features zur Vorhersage eines Modells.

Wir verwenden dazu den `Explainer` aus der `shapiq`-Bibliothek mit dem Index `k-SII` (Shapley Interaction Index).
Dabei legen wir die maximale Interaktionsordnung (`max_order`) auf 3 fest – d.h. es werden Einzel-, Paar-

Die Erklärung erfolgt für einen einzelnen Datenpunkt.


In [3]:
from shapiq import Explainer

explainer = Explainer(
    model=model.predict,
    data=X_train.to_numpy(),
    index="k-SII",
    max_order=3,
)

# Wir erklären den ersten Datenpunkt aus dem Trainingsset
x0 = X_train.to_numpy()[0]
interaction_values = explainer.explain(x0, budget=512)
interaction_values

/Users/samuelhutt/Developer/sep_mlai/sep-aiml-25/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Users/samuelhutt/Developer/sep_mlai/sep-aiml-25/.venv/lib/python3.13/site-packages/shapiq/games/imputer/marginal_imputer.py:103: UserWarning: The sample size is larger than the number of data points in the background set. Reducing the sample size to the number of background samples.
  self.init_background(self.data)
/Users/samuelhutt/Developer/sep_mlai/sep-aiml-25/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Users/samuelhutt/Developer/sep_mlai/sep-aiml-25/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was f

InteractionValues(
    index=k-SII, max_order=3, min_order=0, estimated=True, estimation_budget=512,
    n_players=12, baseline_value=223.595
)

## Koalitionsfindung mit Beam Search

Mit den berechneten Interaktionswerten können wir nun nach optimalen Koalitionen suchen.

Die Funktion `subset_finding` verwendet eine heuristische Beam Search, um aus allen möglichen Feature-Teilgruppen
der Größe `l` die beste und schlechteste Koalition zu finden – also jene mit dem höchsten bzw. niedrigsten
aggregierten Interaktionswert.

Wir wählen hier eine Koalitionsgröße von `l = 3`, also Teilmengen von jeweils 3 Features.


In [4]:
# Import des Koalitionsfindungs-Algorithmus
from shapiq_student.subset_finding import subset_finding

# Suche nach Koalitionen der Größe 3
beam_result = subset_finding(interaction_values=interaction_values, max_size=3)
beam_result

InteractionValues(
    index=k-SII, max_order=3, min_order=3, estimated=True, estimation_budget=None,
    n_players=12, baseline_value=223.595
)

## Analyse der gefundenen Koalitionen

Die Funktion `subset_finding` liefert zwei Koalitionen:

- `s_max`: Die Koalition der Größe `l`, die den höchsten aggregierten Interaktionswert erzielt
- `s_min`: Die Koalition mit dem niedrigsten Wert

Zusätzlich gibt sie auch die zugehörigen Werte `v_max` und `v_min` zurück.

Im Folgenden untersuchen wir die Koalitionen genauer – sowohl in numerischer Form (Index) als auch lesbar mit Spaltennamen.


In [5]:
# Rohausgabe
print("Beste Koalition (Index):", beam_result.s_max)
print("Schlechteste Koalition (Index):", beam_result.s_min)
print("Wert der besten Koalition:", beam_result.v_max)
print("Wert der schlechtesten Koalition:", beam_result.v_min)

# Lesbare Darstellung mithilfe der Spaltennamen
columns = X_train.columns


def readable(S: set[int]) -> set[str]:
    return {columns[i] for i in S}


print("Beste Koalition (Namen):", readable(beam_result.s_max))
print("Schlechteste Koalition (Namen):", readable(beam_result.s_min))

Beste Koalition (Index): {1, 2, 4}
Schlechteste Koalition (Index): {8, 11, 7}
Wert der besten Koalition: 494.598146722589
Wert der schlechtesten Koalition: 207.35762451652712
Beste Koalition (Namen): {'feel_temp', 'temp', 'windspeed'}
Schlechteste Koalition (Namen): {'weekday', 'holiday', 'weather'}


## Vergleich mit Brute-Force

Um die Qualität der gefundenen Koalitionen zu bewerten, vergleichen wir das Ergebnis der heuristischen Beam-Search
(`subset_finding`) mit dem exakten Brute-Force-Verfahren.

Brute-Force durchsucht **alle möglichen Koalitionen der Größe `l`** und wählt exakt die mit dem höchsten bzw. niedrigsten Wert aus.
Da dieser Ansatz exponentiell skaliert, ist er nur für kleine Feature-Anzahlen praktikabel – eignet sich aber ideal zur Verifikation.


In [7]:
# Import für die Laufzeitmessung
import time

# Import des Brute-Force-Vergleichs und Bewertungsfunktion
from shapiq_student.brute_force import brute_force_find_extrema
from shapiq_student.evaluation import evaluate_interaction_coalition

# Vorbereitung: Feature-Indizes, Interaktionswerte und Spaltennamen
features = list(range(interaction_values.n_players))
interactions = interaction_values.dict_values
columns = X_train.columns  # für lesbare Koalitionsnamen


# Bewertungsfunktion für Koalitionen
def evaluate(S: set[int], e: dict[tuple[int, ...], float]) -> float:
    return evaluate_interaction_coalition(S, e, max_order=3)


# Hilfsfunktion zur Darstellung von Koalitionen mit Namen
def readable(S: set[int]) -> str:
    return "{" + ", ".join(columns[i] for i in S) + "}"


# Beam Search mit Zeitmessung
start_beam = time.perf_counter()
beam_result = subset_finding(interaction_values=interaction_values, max_size=3)
end_beam = time.perf_counter()
beam_time = end_beam - start_beam

# Brute-Force mit Zeitmessung
start_brute = time.perf_counter()
S_max_b, val_max_b = brute_force_find_extrema(features, interactions, evaluate, size=3, mode="max")
S_min_b, val_min_b = brute_force_find_extrema(features, interactions, evaluate, size=3, mode="min")
end_brute = time.perf_counter()
brute_time = end_brute - start_brute

# Vergleichsausgabe
print("Beam Search:")
print(f"  Beste Koalition : {readable(beam_result.s_max)} → {beam_result.v_max:.4f}")
print(f"  Schlechteste    : {readable(beam_result.s_min)} → {beam_result.v_min:.4f}")
print(f"  Laufzeit        : {beam_time:.4f} Sekunden\n")

print("Brute-Force:")
print(f"  Beste Koalition : {readable(S_max_b)} → {val_max_b:.4f}")
print(f"  Schlechteste    : {readable(S_min_b)} → {val_min_b:.4f}")
print(f"  Laufzeit        : {brute_time:.4f} Sekunden")

Beam Search:
  Beste Koalition : {temp, feel_temp, windspeed} → 494.5981
  Schlechteste    : {weekday, weather, holiday} → 207.3576
  Laufzeit        : 0.0050 Sekunden

Brute-Force:
  Beste Koalition : {temp, feel_temp, windspeed} → 494.5981
  Schlechteste    : {weekday, weather, holiday} → 207.3576
  Laufzeit        : 0.0600 Sekunden


## Fazit

In diesem Notebook haben wir demonstriert, wie sich mit dem Algorithmus `subset_finding` auf Basis von Beam Search
optimale Feature-Koalitionen identifizieren lassen.

Der Algorithmus erlaubt es, für eine gegebene Koalitionsgröße `l` effizient diejenige Teilmenge von Features zu finden,
die den höchsten bzw. niedrigsten aggregierten Interaktionswert besitzt. Dies erfolgt heuristisch und ist besonders bei
größeren Feature-Räumen deutlich schneller als eine vollständige Brute-Force-Suche.

Ein direkter Vergleich mit dem exakten Brute-Force-Verfahren zeigt, dass die Beam Search sehr gute Näherungen liefert –
bei einem Bruchteil der Laufzeit. Damit eignet sich der Ansatz insbesondere für praxisnahe Anwendungen, bei denen
Interpretierbarkeit und Skalierbarkeit gleichzeitig wichtig sind.

Der Algorithmus lässt sich flexibel auf Interaktionen höherer Ordnung anwenden und stellt eine leistungsfähige Erweiterung
zur Analyse modellbasierter Erklärungen dar.
